# Intraday Mean reversion Steratey

1. Select all stocks near market open, compute rolling mean and standard derivation of last $r$ days close-to-close return. Identify the gapped down when
    - <b>(a)</b> Returns from T-1 Low to T open are lower than $z_{th}$ std
    - <b>(b)</b> Returns from T-1 high to T open are high than $z_{th}$ std
    

2. For each stocks, compute the correlation of 
    - Historically, when <b>(a)</b> happened, correlation of T-1 Low to T Open return vs T Open to Close return 
    - Historically, when <b>(b)</b> happened, correlation of T-1 High to T Open return vs T Open to Close return 

3. Further narrow down the universe by selecting only stocks when correlation from <b>(2)</b> is smaller than $b$

4. Long <b>(a)</b> and short <b>(b)</b>

5. Liquidate all positions at market close

# Preprocess the data

1. We try find all historical days when either <b>(a)</b> or <b>(b)</b> happened

2. Measure the correlation of L2O or H2O against next day intraday returns => beta

3. The beta will be used as further identify which stocks are liekly to mean revert when <b><i>this</i></b> happened 

My Notes:

<b>If the close to open change is driven by entire economic event (e.g. CPI, FF rate), we should avoid to trade on these days. Mean reversion on individual stocks are less liekly to happen</b>

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from utils.performance import *
from utils.data import *
from utils.earnings_calendar import *
from utils.data_helper import *
from utils.logging import get_logger
import utils.format

from pandas.tseries.offsets import BDay
from tqdm import tqdm
from matplotlib.gridspec import GridSpec
from matplotlib import dates as mdates
from account.Futu import *
import time
import warnings
from scipy.stats import pearsonr   
from Strategy import IntradayMeanReversion

logger = get_logger('Intraday Mean reversion Steratey')

# BackTest

## My notes
- We used to backtest on SP500 stocks, now we can expand the strategy to all us traded stocks, we can start with stocks in mid and large cap and check the mean reversion

- use multithread to speedup yahoo finance api

- trade stocks with decent volume

In [14]:
start_date = datetime(2022,6,1)
end_date = add_bday(get_today(), -1)
stock_universe = get_sp500_tickers()

stock_universe = get_us_listed_stocks_table()
stock_universe = stock_universe[stock_universe['marketCap'] > 2e9]
stock_universe = list(stock_universe['symbol'].unique())

In [58]:
# strategy = IntradayMeanReversion()    
# strategy.set_stock_universe(stock_universe)
# strategy.set_start_date(start_date)
# strategy.set_end_date(end_date)
strategy.adv_min = 50e6
strategy.z_threshold = 1
strategy.beta_threshold = 0.5
strategy.z_rolling_windows = 90
strategy.comms_per_round = 4
#strategy.preprocess_data()
strategy.generate_position()
strategy.backtest_summary()

2023-12-28 00:52:43,350 - Intraday Mean Reversion - INFO - Generating positions.....


100%|██████████| 2012/2012 [00:03<00:00, 612.09it/s]


,Intraday Mean Reversion,^SPX,^IXIC
Measure,,,
Cumulative Return,0.695827,1.164224,1.256794
Annualized Return,-0.221063,0.113350,0.172839
Annualized Volatility,0.142642,0.180764,0.232438
Annualized Sharpe Ratio,-1.796676,0.432233,0.592076
Maximum Drawdown,-0.347665,-0.169137,-0.222025


In [ ]:
def plot_z_vs_next(df, title, prev_ret, prev_ret_z):    
    fig = plt.figure(figsize=(8,4))    
    fig.subplots_adjust(hspace=0.5, wspace=0.3)
    fig.suptitle(title, y=1.05, fontsize=20)
    gs = GridSpec(1,2)

    ax = fig.add_subplot(gs[0])
    x = df[prev_ret_z].values
    y = df['intra_ret'].values

    ax.scatter(x, y)
    ax.axvline(0, linestyle='--', color='black')
    ax.axhline(0, linestyle='--', color='black')
    ax.plot(x, np.polyval(np.polyfit(x, y, 1),x), color='black', linewidth=5)
    ax.set_title(f'ZScore ({prev_ret_z}) vs Next Intraday Return')
    ax.grid()
    ax.set_xlabel('Z Score')
    ax.set_ylabel('Next IntraDay Return')

    ax = fig.add_subplot(gs[1])
    x = df[prev_ret].values
    y = df['intra_ret'].values

    ax.scatter(x, y)
    ax.plot(x, np.polyval(np.polyfit(x, y, 1),x), color='black', linewidth=5)
    ax.axvline(0, linestyle='--', color='black')
    ax.axhline(0, linestyle='--', color='black')
    ax.set_title('Prev C2O Ret vs Next Intraday O2C Ret')
    ax.grid()
    ax.set_xlabel(f'Prev {prev_ret}')
    ax.set_ylabel('Next IntraDay Return')
    plt.show()

df_trade_stats = strategy.df_trade_stats
beta_threshold = strategy.beta_threshold

temp = df_trade_stats[df_trade_stats['l2o_beta'] < -beta_threshold]
temp = temp[temp['position'] != 0]
plot_z_vs_next(temp, f'All Time (Traded Names with L2O Beta < -{strategy.beta_threshold})','l2o_ret', 'l2o_ret_z')

temp = df_trade_stats[df_trade_stats['h2o_beta'] < -beta_threshold]
temp = temp[temp['position'] != 0]
plot_z_vs_next(temp, f'All Time (Traded Names with H2O Beta < -{strategy.beta_threshold})','h2o_ret', 'h2o_ret_z')

# for q in sorted(df_stats['quarter'].unique(), reverse=True):
#     temp = df_stats[df_stats['quarter'] == q]
#     plot_z_vs_next(temp, q)


In [ ]:
# Get latest return (i.e. returns for yesterday)
df_trade_stats = strategy.df_trade_stats
latest = df_trade_stats[df_trade_stats['date'] == end_date]

ret = (latest['weight'] * latest['intra_ret']).sum()
latest['correct_dir'] = np.where(latest['position'] != 0, np.sign(latest['position']) == np.sign(latest['intra_ret']), np.nan)    
correct_dir_pct = latest['correct_dir'].sum() / (1 * (latest['position'] != 0).sum())
no_of_long = len(latest[latest['position'] > 0])
no_of_short = len(latest[latest['position'] < 0])

logger.info('Intra-day return on {} is {:.2f}%, pct of hit is {:.2f}%'.format(end_date.strftime('%Y-%m-%d'), ret*100, correct_dir_pct*100))
logger.info('# of Longs:  {:.0f}'.format(no_of_long))
logger.info('# of Shorts: {:.0f}'.format(no_of_short))
display(latest)

# Parameters calibration

- Calibrate the model parameters by maximize the sharpe of the strategy

- Z threshold as 0.4 is found to be the best to maximize the Sharpe Ratio, but the number of daily trades are really high, considering the transaction costs, it might be lower

In [ ]:
# start_date = datetime(2021,1,1)
# end_date = add_bday(get_today(), -1)
# stock_universe = get_sp500_tickers()

# strategy = IntradayMeanReversion()    
# strategy.set_stock_universe(stock_universe)
# strategy.set_start_date(start_date)
# strategy.set_end_date(end_date)
# strategy.preprocess_data() 

# fig = plt.figure(figsize=(8,4))    
# fig.subplots_adjust(hspace=0.5, wspace=0.3)
# gs = GridSpec(1,2)

# beta = np.arange(0,1,0.1)
# z_th = np.arange(0,2,0.1)
# beta_sr = []
# z_th_sr = []

# for b in beta:
#     strategy.__init__()
#     strategy.set_beta_threshold(b)
#     strategy.generate_position(skip_trade_stats=True)
#     strategy.generate_backtest_return()
#     port_ret = strategy.port_ret
#     beta_sr.append(annualized_sharpe_ratio(port_ret))    

# for z in z_th:    
#     strategy.__init__()
#     strategy.set_z_threshold(z)
#     strategy.preprocess_data()
#     strategy.generate_position(skip_trade_stats=True)
#     strategy.generate_backtest_return()
#     port_ret = strategy.port_ret
#     z_th_sr.append(annualized_sharpe_ratio(port_ret))

# ax = fig.add_subplot(gs[0])
# ax.plot(beta, beta_sr)
# ax.set_ylabel('Sharpe Ratio')
# ax.set_xlabel('Beta threshold')
# ax.grid()
# ax.set_title('Beta threshold vs Sharpe')

# ax = fig.add_subplot(gs[1])
# ax.plot(z_th, z_th_sr) 
# ax.set_ylabel('Sharpe Ratio')
# ax.set_xlabel('Z threshold')
# ax.grid()
# ax.set_title('Z threshold vs Sharpe')

# Execute the strategy

1. Get real-time open price

2. Follows the strategy to derive the positions

3. Convert the position to shares

4. Send the orders

5. Liquidate all orders by end of day

In [62]:
#stock_universe = get_sp500_tickers()
stock_universe = get_us_listed_stocks_table()
stock_universe = stock_universe[stock_universe['marketCap'] > 2e9]
stock_universe = list(stock_universe['symbol'].unique())

capital = 20000
today = add_bday(add_bday(get_today(), -1), 1)

In [20]:
strategy = IntradayMeanReversion()    
strategy.set_stock_universe(stock_universe)
strategy.set_capital(capital)
strategy.actual_trade(today)

2023-12-27 23:30:00,935 - Intraday Mean Reversion - INFO - ============================== Intraday Mean Reversion Setup ==============================
2023-12-27 23:30:00,936 - Intraday Mean Reversion - INFO - capital:                     20000
2023-12-27 23:30:00,936 - Intraday Mean Reversion - INFO - is_backtest:                 False
2023-12-27 23:30:00,937 - Intraday Mean Reversion - INFO - open_period_to_use_close:    20
2023-12-27 23:30:00,937 - Intraday Mean Reversion - INFO - z_rolling_windows:           60
2023-12-27 23:30:00,937 - Intraday Mean Reversion - INFO - z_threshold:                 0.4
2023-12-27 23:30:00,938 - Intraday Mean Reversion - INFO - corr_rolling_windows:        10
2023-12-27 23:30:00,938 - Intraday Mean Reversion - INFO - beta_threshold:              0.3
2023-12-27 23:30:00,939 - Intraday Mean Reversion - INFO - min_trades:                  5
2023-12-27 23:30:00,939 - Intraday Mean Reversion - INFO - max_trades:                  5
2023-12-27 23:30:00,939 

[*********************100%***********************]  2012 of 2012 completed

2 Failed downloads:
- BRK/A: No timezone found, symbol may be delisted
- BRK/B: No timezone found, symbol may be delisted


2023-12-27 23:32:09,880 - Intraday Mean Reversion - CRITICAL - Actual Trade Mode: After 20 mins of market open, use open price
100%|██████████| 2012/2012 [00:43<00:00, 45.76it/s]
2023-12-27 23:32:56,495 - Intraday Mean Reversion - INFO - Generating positions.....
100%|██████████| 2012/2012 [00:02<00:00, 683.57it/s]


In [21]:
today_stats = strategy.df_trade_stats
today_stats

,date,symbol,signal,open,close,last_high,last_low,c2c_ret_mean,c2c_ret_std,l2o_ret,l2o_ret_z,l2o_beta,h2o_ret,h2o_ret_z,h2o_beta,intra_ret,beta_used,total_trades,weight,position,shares,allocated_captial
0,2023-12-27,KT,1,13.430000,13.670100,13.840000,13.790000,0.001340,0.012400,-0.026106,-2.213374,-0.597260,-0.029624,-2.497117,-0.525905,0.017878,-0.597260,5,0.2,297.840648,298.0,4000.0
1,2023-12-27,NTES,1,88.519997,88.309998,94.459999,91.160004,-0.000898,0.027949,-0.028960,-1.004046,-0.817948,-0.062884,-2.217809,-0.527673,-0.002372,-0.817948,5,0.2,45.187530,45.0,4000.0
2,2023-12-27,OXLCL,1,22.940001,23.530001,23.100000,22.997999,0.000610,0.005874,-0.002522,-0.533224,-0.828725,-0.006926,-1.282999,-0.790588,0.025719,-0.828725,5,0.2,174.367912,174.0,4000.0
3,2023-12-27,UCBIO,-1,23.500000,22.815001,23.299999,22.549999,0.000634,0.011480,0.042129,3.614560,0.197713,0.008584,0.692487,-0.578530,-0.029149,-0.578530,5,-0.2,-170.212766,-170.0,-4000.0
4,2023-12-27,VLYPO,-1,23.940001,23.940001,23.799999,23.719999,0.001229,0.010586,0.009275,0.760073,0.602873,0.005882,0.439612,-0.589204,0.000000,-0.589204,5,-0.2,-167.084374,-167.0,-4000.0


## Execute the orders to open the position

In [26]:
strategy.enter_strategy(is_test=False)
strategy.execute_orders

2023-12-27 23:37:21,125 | 5330 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=26, host=127.0.0.1, port=11111, user_id=18214795


2023-12-27 23:37:21,138 - Futu - INFO - 20 Positions: US.TSLA240105C300000, US.TSLA240105C290000, US.TSLA231229C300000, US.TSLA231229C290000, US.TSLA231229C280000, US.AZN, US.XEL, US.VOE, US.VBR, US.TSLA, US.SPY, US.QQQ, US.OXLCL, US.NVDA, US.NTES, US.KT, US.EWY, US.CIB, US.BRK.B, US.AAPL


2023-12-27 23:37:21,140 | 5330 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=26


2023-12-27 23:37:21,140 - Futu - INFO - US.KT Already Traded: skip
2023-12-27 23:37:21,140 - Futu - INFO - US.NTES Already Traded: skip
2023-12-27 23:37:21,141 - Futu - INFO - US.OXLCL Already Traded: skip


2023-12-27 23:37:21,145 | 5330 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=27, host=127.0.0.1, port=11111, user_id=18214795


2023-12-27 23:37:21,371 - Futu - ERROR - place_order_error: 该产品现不支持卖空


2023-12-27 23:37:21,373 | 5330 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=27
2023-12-27 23:37:24,384 | 5330 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=28, host=127.0.0.1, port=11111, user_id=18214795


2023-12-27 23:37:24,603 - Futu - ERROR - place_order_error: 当前产品已被禁止卖出开仓。


2023-12-27 23:37:24,604 | 5330 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=28
[]


[]

## Close out all intraday positions

In [63]:
strategy = IntradayMeanReversion()
strategy.set_start_date(today)
strategy.set_end_date(today)
strategy.exit_strategy(exit_time='04:50:00', is_test=False)
strategy.exit_orders

2023-12-28 03:19:33,983 | 20917 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=18214795


2023-12-28 03:19:34,015 - Futu - INFO - 20 Positions: US.TSLA240105C300000, US.TSLA240105C290000, US.TSLA231229C300000, US.TSLA231229C290000, US.TSLA231229C280000, US.AZN, US.XEL, US.VOE, US.VBR, US.TSLA, US.SPY, US.QQQ, US.OXLCL, US.NVDA, US.NTES, US.KT, US.EWY, US.CIB, US.BRK.B, US.AAPL


2023-12-28 03:19:34,017 | 20917 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1


2023-12-28 03:19:34,261 - Intraday Mean Reversion - INFO - Total 6 stocks to close....
2023-12-28 03:19:34,261 - Intraday Mean Reversion - INFO - ['US.AZN' 'US.CIB' 'US.KT' 'US.NTES' 'US.OXLCL' 'US.XEL']


2023-12-28 04:50:00,638 | 20917 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=2, host=127.0.0.1, port=11111, user_id=18214795


...

2023-12-28 04:50:14,167 | 20917 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=7
2023-12-28 04:50:17,178 | 20917 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=8, host=127.0.0.1, port=11111, user_id=18214795


2023-12-28 04:50:17,472 - Futu - INFO - Placed Order: {'code': 'US.CIB', 'price': 1, 'qty': 133.0, 'trd_side': 'SELL', 'order_type': 'MARKET', 'market': 'US', 'trd_env': 'REAL'}


2023-12-28 04:50:17,475 | 20917 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=8


2023-12-28 04:50:20,503 - Futu - INFO - Saving 2023-12-27 orders for strategy Intraday Mean Reversion....
2023-12-28 04:50:20,536 - Intraday Mean Reversion - ERROR - Trying to store a string with len [57] in [values_block_3] column but
this column has a limit of [32]!
Consider using min_itemsize to preset the sizes on these columns


AttributeError: 'IntradayMeanReversion' object has no attribute 'exit_orders'

# Order History

In [ ]:
Futu().order_history(start_date=today, end_date=today + BDay(1))